In [1]:
#!/home/ektov1-av_ca-sbrf-ru/bin/python35
import os
import sys
curruser = os.environ.get('USER')

# sys.path.insert(0, '/opt/workspace/{user}/system/support_library/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/libs/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/system/labdata/lib/'.format(user=curruser))


sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))
sys.path.insert(0, './src')

#import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(filename='./__move_dbs__.log',level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger = logging.getLogger(__name__)


import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
from datetime import datetime

from transliterate import translit

from spark_connector import SparkConnector
from sparkdb_loader import spark
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.utils import AnalysisException


import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load

# sing = tendo.singleton.SingleInstance()

# os.chdir('/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/Clickstream_Analytics/AutoUpdate/')
# os.chdir('/opt/workspace/{}/notebooks/clickstream/AutoUpdate/'.format(curruser))


##-----------------------------------------
conn_schema = 'sbx_team_digitcamp' #'dl_t_team_ds_kb_sme' 'dl_t_team_speech_analytics'
new_schema = 'sbx_t_team_cvm'
ga_schema = 'sklod_external_google_analytics'
##-----------------------------------------


def show(self, n=10):
    return self.limit(n).toPandas()

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

def essense(channel: str, prod_cd: str):
    message = "{}: {} retargeting".format(channel, prod_cd)
    return message

essense_udf = f.udf(essense, StringType())

pyspark.sql.dataframe.DataFrame.show = show

def print_and_log(message: str):
    print(message)
    logger.info(message)
    return None


In [2]:
print("### Starting spark context. Run!")

sp = spark(schema=conn_schema,
           dynamic_alloc=False,
           numofinstances=10, 
           numofcores=8,
           kerberos_auth=True,
           process_label="SAS_"
           )
hive = sp.sql

print(sp.sc.version)

### Starting spark context. Run!
2.4.0.cloudera2


In [7]:
hive.sql(''' SELECT * FROM sbx_team_digitcamp.ma_dict_v_product_dict
             where crm_product_id = '1-2AZ4PW6S' ''').show()

,ID,CRM_PRODUCT_ID,PRODUCT_CRM_NM,CRM_ACTIVE_FLG,PRODUCT_CD_KSB,PRODUCT_CD_RGS,PRODUCT_CD_MMB,PRODUCT_CD_ASUP,PRODUCT_SHORT_NM,PRODUCT_FULL_NM,...,CHANNEL_MMB_CKR,CHANNEL_MMB_MKK,CHANNEL_DZO,AGG_FORMULA_KSB,AGG_FORMULA_RGS,CNT_DAY_DEAL,PRODUCT_SUBGROUP,PRODUCT_GROUP,PRODUCT_TYPE,PRODUCT_PRIORITY
0,4344,1-2AZ4PW6S,2GIS-Вывеска,1,2170,2170,2GIS_VIVESKA,two_gis,2GIS-Вывеска,2GIS-Вывеска,...,1,0,0,0,0,30.0,2GIS,Экосистема,Экосистема,0


In [35]:
table_name = 'telemarketing_cell2inn' #'MA_CMDM_MA_PRODUCT_OFFER'

In [33]:
res = hive.sql("describe extended {}.{}".format(conn_schema,table_name)).collect()
len([item.asDict() for item in res if item.asDict()['col_name'] =='# Partition Information']) > 0

True

In [14]:
hive.sql('''ALTER TABLE {}.{} 
            DROP IF EXISTS PARTITION(host_prod_id='__HIVE_DEFAULT_PARTITION__')'''.format(conn_schema,table_name))

DataFrame[]

In [49]:
parts = hive.sql("show partitions {}.{}".format(conn_schema,table_name)).collect()
max_part = sorted(parts,reverse=True)[0]['partition']
extract_date=re.compile("\d{4}\-\d{2}\-\d{2}")
ext = extract_date.search(max_part)
try:
    max_trunc_dt = ext.group(0)
except:
    max_trunc_dt = None
max_trunc_dt

'2021-04-01'

In [51]:
sql = '''select max(creation_dttm) from {}.{} where creation_month = '{}' '''.format(conn_schema,table_name,max_trunc_dt)
max_dt = hive.sql(sql).collect()
max_dt

[Row(max(creation_dttm)=datetime.datetime(2021, 4, 19, 6, 51, 14))]

In [3]:
# list of tables without partitioned column
dbslst =\
[
#     'MA_CMDM_MA_DEAL',
#     'GA_MA_CUSTOMER_SBBOL',
#     'GA_MA_USER_PROFILE_SBBOL',
#     'MA_MMB_OFFER_NONTOP' ,
#     'OFFER_PRIORITY',
#     'MA_MMB_PRODUCT_DICT',
#     'UNIFIED_CUSTOMER' ,
#     'MA_CMDM_MA_AGREEMENT',
#     'SME_INT_PIPELINE_BASE',
#     'SME_CDM_V_RESPONSE',
#     'SME_EMAIL_TO_SENDSAY',
#     'SME_INTEGRATION_SMS_QUERY',
#     'SME_CDM_CI_CELL_PACKAGE',
#      'mw_atb_segmen_prod_group'    
#     'CALL_TASK_TO_SENDSAY',
#     'RESP_INN_KPP_CAMP',
#     'RESP_SMS_INN_KPP_CAMP',    
#     'RESP_INN_KPP_PROBA',
#     'RESP_INN_KPP_CAMP_PROBA',
#     'RESP_INN_KPP_CAMP_EMAIL_PROBA',
#     'CALLS_UUP_DETAIL',
#     'SAS_A1_21_TE_COL_N',
#     'SAS_Y1_01_KORSAKOV_PRIORITY',
#     'AIST_INTERACTION_HISTORY',
#     'MA_TASK'
#     'FV_CKR_UUP_PP',
#     'FV_CKR_UUP_VAL'
#       'MA_MMB_OFFER_NONTOP',
#       'OFFER_PRIORITY'
      'INTERNAL_CLICKSTREAM_SITE_CORP',
      'INTERNAL_CLICKSTREAM_INN_COOKIE',
      'INTERNAL_PANHASH_COOKIES',
      'ma_product_dict',
      'ma_stuff',
      'ma_mmb_offer_nontop',
      'offer_priority'
    
]

# list of partitioned tables
partdbs = \
[    
#     ('GA_SITE_ALL_PRODS', 'ctl_loading', 'bigint')
#     ('GA_CID_SBBOL_INN', 'ctl_loading', 'bigint'),
#     ('GA_ALL_SCENARIOS_HIST', 'ctl_loading', 'bigint'),
#     ('RESP_SMS_INN_KPP_CAMP','last_day', 'string'),
#     ('RESP_INN_KPP_CAMP', 'LAST_DAY', 'string')
]

## Replication of simple nonpartitioned tables

In [5]:
import subprocess

In [4]:
for tbl in dbslst:
    print_and_log("### start replication of table: {}".format(tbl))
    hive.sql("drop table if exists {new_schema}.{tbl}".format(new_schema=new_schema, tbl=tbl))
    query = "create table {new_schema}.{tbl} select * from {schema}.{tbl}".format(schema=conn_schema,
                                                                                  new_schema=new_schema,
                                                                                  tbl=tbl)
    hive.sql(query)
    subprocess.call(['hdfs', 'dfs', '-chmod', '-R', '777', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format(tbl.lower())])
    print_and_log("### table: {} has been succesfully replicated".format(tbl))

### start replication of table: INTERNAL_CLICKSTREAM_SITE_CORP
### table: INTERNAL_CLICKSTREAM_SITE_CORP has been succesfully replicated
### start replication of table: INTERNAL_CLICKSTREAM_INN_COOKIE
### table: INTERNAL_CLICKSTREAM_INN_COOKIE has been succesfully replicated
### start replication of table: INTERNAL_PANHASH_COOKIES
### table: INTERNAL_PANHASH_COOKIES has been succesfully replicated
### start replication of table: ma_product_dict
### table: ma_product_dict has been succesfully replicated
### start replication of table: ma_stuff
### table: ma_stuff has been succesfully replicated
### start replication of table: ma_mmb_offer_nontop
### table: ma_mmb_offer_nontop has been succesfully replicated
### start replication of table: offer_priority
### table: offer_priority has been succesfully replicated


## Replication of partitioned tables

In [3]:
conn_schema = 'sbx_team_digitcamp' #'dl_t_team_ds_kb_sme' 'dl_t_team_speech_analytics'
new_schema = 'sbx_t_team_cvm'

In [4]:
hive.setConf("hive.exec.dynamic.partition","true")
hive.setConf("hive.exec.dynamic.partition.mode","nonstrict")
hive.setConf("hive.enforce.bucketing","false")
hive.setConf("hive.enforce.sorting","false")

In [14]:
part_cols  =  [('interaction_end_month',),
               ('string',)
              ]
bucket_col = ["end_plan_dt", "end_fact_dt", "inn"]
bucket_num = 125

table_name = 'AIST_INTERACTION_HISTORY'
part_tbl = "tmp_"+table_name

In [16]:
sdf = hive.sql("select * from {}.{}".format(conn_schema,table_name))
# sdf = sp.get_oracle(OracleDB('iskra4'), '''( select /*+ parallel (12) */ * from MA_CMDM_MA_DEAL 
#                                              where create_dt <= to_timestamp('2020-08-01 00:00:00', 'yyyy-mm-dd hh24:mi:ss')
#                                             )''')
sdf = sdf.withColumn('interaction_end_month',f.trunc(f.to_date('interaction_end_date'),'MONTH').cast(StringType()))
# strdate = datetime.strftime(datetime.now(), format='%Y.%d.%m')
# currdate = datetime.strptime(strdate,'%Y.%d.%m')
# sdf = sdf.withColumn('load_dttm',f.lit(currdate).cast(TimestampType()))
# sdf.registerTempTable("tmp_insight")
# sdf.count()

## Create Repartitioned table based on the origin table in the same scheme (with '_NEW' suffix)

In [ ]:
print_and_log("### start replication of table: {}".format(table_name))

hive.sql("drop table if exists {schema}.{tbl} purge".format(schema=conn_schema, tbl=table_name+'_NEW'))
#     sdf = hive.sql("select * from {}.{}".format(conn_schema,tbl))
insert = ', '.join(["{} {}".format(col, _type) for col, _type in sdf.dtypes if col.lower() not in part_cols[0]])
part_tupl_lst = ', '.join(["{} {}".format(col, _type) for col, _type in list(zip(part_cols[0],part_cols[1]))])
part_col_lst  = ', '.join([col for col in part_cols[0]])
bucket_cols = ', '.join([col for col in bucket_col])   

hive.sql('''create table {schema}.{tbl} (
                                         {fields}
                                            )
             PARTITIONED BY ({part_col_lst})
             CLUSTERED BY ({bucket_cols}) INTO {bucket_num} BUCKETS STORED AS PARQUET
         '''.format(schema=conn_schema,
                    tbl=table_name+'_NEW',
                    fields=insert,
                    part_col_lst=part_tupl_lst,
                    bucket_num=bucket_num,
                    bucket_cols=bucket_cols)
        )
    
colsinhive = hive.sql("select * from {}.{}".format(conn_schema,table_name+'_NEW')).columns    
sdf = sdf.select(*colsinhive)
sdf.registerTempTable(part_tbl)

hive.sql("""
insert into table {schema}.{tbl}
partition({part_col})
select * from {tmp_tbl}
cluster by ({bucket_cols})
""".format(schema=conn_schema,
           tbl=table_name+'_NEW',
           tmp_tbl=part_tbl,
           part_col=part_col_lst,
           bucket_cols=bucket_cols)
        )    

subprocess.call(['hdfs', 'dfs', '-chmod', '-R', '777', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format(table_name+'_NEW'.lower())])

print_and_log("### table: {} has been succesfully replicated".format(tbl))    

## Recplication between different schemes 

In [36]:
print_and_log("### start replication of table: {}".format(table_name))

hive.sql("drop table if exists {new_schema}.{tbl} purge".format(new_schema=new_schema, tbl=table_name))
#     sdf = hive.sql("select * from {}.{}".format(conn_schema,tbl))
insert = ', '.join(["{} {}".format(col, _type) for col, _type in sdf.dtypes if col.lower() not in part_cols[0]])
part_tupl_lst = ', '.join(["{} {}".format(col, _type) for col, _type in list(zip(part_cols[0],part_cols[1]))])
part_col_lst  = ', '.join([col for col in part_cols[0]])
bucket_cols = ', '.join([col for col in bucket_col])

#     hive.sql('''create table {schema}.{tbl} (
#                                              {fields}
#                                                 )
#                  PARTITIONED BY ({part_col} {col_type})
#              '''.format(schema=new_schema,
#                         tbl=tbl, 
#                         fields=insert, 
#                         part_col=part_col,
#                         col_type=_type)
#             )    

hive.sql('''create table {schema}.{tbl} (
                                         {fields}
                                            )
             PARTITIONED BY ({part_col_lst})
             CLUSTERED BY ({bucket_cols}) INTO {bucket_num} BUCKETS STORED AS PARQUET
         '''.format(schema=new_schema,
                    tbl=table_name,
                    fields=insert,
                    part_col_lst=part_tupl_lst,
                    bucket_num=bucket_num,
                    bucket_cols=bucket_cols)
        )
    
colsinhive = hive.sql("select * from {}.{}".format(new_schema,table_name)).columns    
sdf = sdf.select(*colsinhive)
sdf.registerTempTable(part_tbl)

hive.sql("""
insert into table {schema}.{tbl}
partition({part_col})
select * from {tmp_tbl}
cluster by ({bucket_cols})
""".format(schema=new_schema,
           tbl=table_name,
           tmp_tbl=part_tbl,
           part_col=part_col_lst,
           bucket_cols=bucket_cols)
        )    

#     hive.sql("""
#     insert into table {new_schema}.{tbl}
#     partition({part_col}) 
#     select * from {schema}.{tbl}
#     cluster by ({bucket_cols}) 
#     """.format(schema=conn_schema,
#                new_schema=new_schema, 
#                tbl=tbl,
#                part_col=part_col,
#                bucket_cols=bucket_cols)
#             )

#     hive.sql("""
#     insert into table {new_schema}.{tbl}
#     partition({part_col}) 
#     select * from {tmp_tbl}
#     distribute by {part_col}
#     """.format(new_schema=new_schema, 
#                tbl=tbl,
#                tmp_tbl="tmp_insight",
#                part_col=part_col)
#             )

subprocess.call(['hdfs', 'dfs', '-chmod', '-R', '777', "hdfs://clsklsbx/user/team/team_cvm/hive/{}".format(tbl.lower())])

print_and_log("### table: {} has been succesfully replicated".format(tbl))    

### start replication of table: MA_CMDM_MA_TASK
### table: MA_CMDM_MA_TASK has been succesfully replicated
